In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
from IPython.display import clear_output, display, Math, Latex
from atlas_ml import *

In [ ]:
dataset = pd.read_csv('Datasets/Classification/Titanic/train.csv')
dataset = dataset.fillna(0)
data = dataset.values.T

In [ ]:
pd.read_csv('Datasets/Classification/Titanic/train.csv')

In [ ]:
def get_title(name):
    if re.search('Mr\.', name):
        return 1
    elif re.search('Mrs\.', name):
        return 2
    elif re.search('Miss\.', name):
        return 3
    else:
        return 4

In [ ]:
get_title('Braund, Mr Owen Harris')

In [ ]:
Pass_id = data[0]
Y = one_hot(np.expand_dims(data[1],0),2)
Pclass = data[2]
Title = [get_title(i) for i in data[3]]
Sex = [1*int(i=='male') for i in data[4] ] #Source of error because NaN values are default assigned 0.
Age = data[5]
SibSp = data[6]
Parch = data[7]
#Ticket = [i]  -> probably irrelevant or redundant with class, fare etc
Fare = data[9]
#Cabin = [int(i[])] -> too many Nan
Embarked = [1*int(i=='S')+2*int(i=='C')+3*int(i=='Q') for i in data[11]]

In [ ]:
# degree of polynomial features
d = 1

In [ ]:
X = np.vstack([normalize(i) for i in [Pclass,Title,Sex,Age,SibSp,Parch,Fare,Embarked]])
size = X.shape[1]
#X_train = X[:,:int(size*0.9)].astype('Float64')
#Y_train = Y[:,:int(size*0.9)].astype('Float64')
X_train = X[:,int(size*0.1):].astype('Float64')
Y_train = Y[:,int(size*0.1):].astype('Float64')
#X_train = X.astype('Float64')
#Y_train = Y.astype('Float64')

PX_train = (polynom_features(X_train,d))
#X_val = X[:,int(size*0.9):].astype('Float64')
#Y_val = Y[:,int(size*0.9):].astype('Float64')
X_val = X[:,:int(size*0.1)].astype('Float64')
Y_val = Y[:,:int(size*0.1)].astype('Float64')

PX_val = (polynom_features(X_val,d))


In [ ]:
n_in = np.shape(PX_train)[0]
n_out = np.shape(Y_train)[0]
log_reg = Logistic(n_in,n_out,CE_loss)

In [ ]:
n_epochs = 150
lr = 0.01
lr_decay = 1
batch_size = 16
beta = 0.99

#regularization parameter labmda
reg_lamda = 0

In [ ]:
train(log_reg, PX_train, Y_train, PX_val, Y_val, model_accuracy, n_epochs, batch_size, lr, lr_decay, beta, reg_lamda)

In [ ]:
class HL_net:
    def __init__(self, X_size, Y_size, lossfn):
        self.L1 = layer(X_size, 32, relu)
        self.L2 = layer(32, 16, relu)        
        self.L3 = layer(16, Y_size, softmax)
        self.lossfn = lossfn()
        
    def f_pass(self, X):
        A1 = self.L1.forward(X)
        A2 = self.L2.forward(A1)
        A3 = self.L3.forward(A2)
        self.H = A3
        return self.H
    
    def back_prop(self,X,Y, batch_size,reg_lambda):
        m = batch_size
        
        self.loss = self.lossfn.get_loss(self.H,Y)
        dL_dZ = self.lossfn.diff(self.H,Y)
        
        self.L3.out_grad(dL_dZ, self.L2.A, m)
        self.L2.grad(self.L3.dZ, self.L3.W, self.L1.A, m)
        self.L1.grad(self.L2.dZ, self.L2.W, X, m)
    
    def optim(self, lr, beta):
        self.L1.step(lr, beta)
        self.L2.step(lr, beta)
        self.L3.step(lr, beta)

In [ ]:
n_in = np.shape(X_train)[0]
n_out = np.shape(Y_train)[0]

#This is the new 'model', so bin_class will be passed to train()
bin_clas = HL_net(n_in,n_out,CE_loss)

In [ ]:
n_epochs = 500
lr = 0.005
lr_decay = 0.9
batch_size = 16
beta = 0.99

#regularization parameter labmda
reg_lamda = 0.1

In [ ]:
train(bin_clas, X_train, Y_train, X_val, Y_val, model_accuracy, n_epochs, batch_size, lr, beta, reg_lamda)

In [ ]:
dataset = pd.read_csv('Datasets/Classification/Titanic/test.csv')
dataset = dataset.fillna(0)
data = dataset.values.T
Pass_id = data[0]
Pclass = data[1]
Title = [get_title(i) for i in data[2]]
Sex = [1*int(i=='male') for i in data[3] ]
Age = data[4]
SibSp = data[5]
Parch = data[6]
#Ticket = [i]  -> probably irrelevant or redundant with class, fare etc
Fare = data[8]
#Cabin = [int(i[])] -> too many Nan
Embarked = [1*int(i=='S')+2*int(i=='C')+3*int(i=='Q') for i in data[10]]

In [ ]:
X_test = np.vstack([normalize(i) for i in [Pclass,Title,Sex,Age,SibSp,Parch,Fare,Embarked]]).astype('Float64')
PX_test = polynom_features(X_test,d)
H = inv_one_hot(bin_clas.f_pass(PX_test))

In [ ]:
submission = pd.DataFrame({'PassengerId':Pass_id.flatten(),'Survived':H.flatten().astype('int')})

In [ ]:
submission.to_csv('submission.csv', index=False)